# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f174cd9abe0>
├── 'a' --> tensor([[-0.5832, -0.0600,  0.9575],
│                   [ 1.5573, -0.6144, -1.4686]])
└── 'x' --> <FastTreeValue 0x7f174ce0e580>
    └── 'c' --> tensor([[ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.2433e-03],
                        [-8.4500e-01, -2.5485e-01,  8.8289e-01,  7.3123e-02],
                        [ 1.1678e+00,  2.0142e+00,  1.0518e+00,  9.9192e-02]])

In [4]:
t.a

tensor([[-0.5832, -0.0600,  0.9575],
        [ 1.5573, -0.6144, -1.4686]])

In [5]:
%timeit t.a

64.9 ns ± 0.0263 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f174cd9abe0>
├── 'a' --> tensor([[ 0.0595, -0.6951, -1.3826],
│                   [-0.3141,  1.0505, -1.0110]])
└── 'x' --> <FastTreeValue 0x7f174ce0e580>
    └── 'c' --> tensor([[ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.2433e-03],
                        [-8.4500e-01, -2.5485e-01,  8.8289e-01,  7.3123e-02],
                        [ 1.1678e+00,  2.0142e+00,  1.0518e+00,  9.9192e-02]])

In [7]:
%timeit t.a = new_value

66 ns ± 0.333 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5832, -0.0600,  0.9575],
               [ 1.5573, -0.6144, -1.4686]]),
    x: Batch(
           c: tensor([[ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.2433e-03],
                      [-8.4500e-01, -2.5485e-01,  8.8289e-01,  7.3123e-02],
                      [ 1.1678e+00,  2.0142e+00,  1.0518e+00,  9.9192e-02]]),
       ),
)

In [10]:
b.a

tensor([[-0.5832, -0.0600,  0.9575],
        [ 1.5573, -0.6144, -1.4686]])

In [11]:
%timeit b.a

61.2 ns ± 0.0378 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2005, -0.2094, -1.9639],
               [-1.0155,  1.1024,  0.4530]]),
    x: Batch(
           c: tensor([[ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.2433e-03],
                      [-8.4500e-01, -2.5485e-01,  8.8289e-01,  7.3123e-02],
                      [ 1.1678e+00,  2.0142e+00,  1.0518e+00,  9.9192e-02]]),
       ),
)

In [13]:
%timeit b.a = new_value

499 ns ± 0.118 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

838 ns ± 0.76 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 4.92 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 412 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 320 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f174cd947f0>
├── 'a' --> tensor([[[-0.5832, -0.0600,  0.9575],
│                    [ 1.5573, -0.6144, -1.4686]],
│           
│                   [[-0.5832, -0.0600,  0.9575],
│                    [ 1.5573, -0.6144, -1.4686]],
│           
│                   [[-0.5832, -0.0600,  0.9575],
│                    [ 1.5573, -0.6144, -1.4686]],
│           
│                   [[-0.5832, -0.0600,  0.9575],
│                    [ 1.5573, -0.6144, -1.4686]],
│           
│                   [[-0.5832, -0.0600,  0.9575],
│                    [ 1.5573, -0.6144, -1.4686]],
│           
│                   [[-0.5832, -0.0600,  0.9575],
│                    [ 1.5573, -0.6144, -1.4686]],
│           
│                   [[-0.5832, -0.0600,  0.9575],
│                    [ 1.5573, -0.6144, -1.4686]],
│           
│                   [[-0.5832, -0.0600,  0.9575],
│                    [ 1.5573, -0.6144, -1.4686]]])
└── 'x' --> <FastTreeValue 0x7f174cd942e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 38.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1724299cd0>
├── 'a' --> tensor([[-0.5832, -0.0600,  0.9575],
│                   [ 1.5573, -0.6144, -1.4686],
│                   [-0.5832, -0.0600,  0.9575],
│                   [ 1.5573, -0.6144, -1.4686],
│                   [-0.5832, -0.0600,  0.9575],
│                   [ 1.5573, -0.6144, -1.4686],
│                   [-0.5832, -0.0600,  0.9575],
│                   [ 1.5573, -0.6144, -1.4686],
│                   [-0.5832, -0.0600,  0.9575],
│                   [ 1.5573, -0.6144, -1.4686],
│                   [-0.5832, -0.0600,  0.9575],
│                   [ 1.5573, -0.6144, -1.4686],
│                   [-0.5832, -0.0600,  0.9575],
│                   [ 1.5573, -0.6144, -1.4686],
│                   [-0.5832, -0.0600,  0.9575],
│                   [ 1.5573, -0.6144, -1.4686]])
└── 'x' --> <FastTreeValue 0x7f1724299a60>
    └── 'c' --> tensor([[ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.2433e-03],
                        [-8.4500e-01, -2.5485e-01,  8.8289e-01,

In [23]:
%timeit t_cat(trees)

30.2 µs ± 47.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.2433e-03],
                       [-8.4500e-01, -2.5485e-01,  8.8289e-01,  7.3123e-02],
                       [ 1.1678e+00,  2.0142e+00,  1.0518e+00,  9.9192e-02]],
              
                      [[ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.2433e-03],
                       [-8.4500e-01, -2.5485e-01,  8.8289e-01,  7.3123e-02],
                       [ 1.1678e+00,  2.0142e+00,  1.0518e+00,  9.9192e-02]],
              
                      [[ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.2433e-03],
                       [-8.4500e-01, -2.5485e-01,  8.8289e-01,  7.3123e-02],
                       [ 1.1678e+00,  2.0142e+00,  1.0518e+00,  9.9192e-02]],
              
                      [[ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.2433e-03],
                       [-8.4500e-01, -2.5485e-01,  8.8289e-01,  7.3123e-02],
                       [ 1.1678e+00,  2.0142e+00,  1.0518e+00,  9.9192e-02]],
      

In [26]:
%timeit Batch.stack(batches)

77.1 µs ± 96.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.2433e-03],
                      [-8.4500e-01, -2.5485e-01,  8.8289e-01,  7.3123e-02],
                      [ 1.1678e+00,  2.0142e+00,  1.0518e+00,  9.9192e-02],
                      [ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.2433e-03],
                      [-8.4500e-01, -2.5485e-01,  8.8289e-01,  7.3123e-02],
                      [ 1.1678e+00,  2.0142e+00,  1.0518e+00,  9.9192e-02],
                      [ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.2433e-03],
                      [-8.4500e-01, -2.5485e-01,  8.8289e-01,  7.3123e-02],
                      [ 1.1678e+00,  2.0142e+00,  1.0518e+00,  9.9192e-02],
                      [ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.2433e-03],
                      [-8.4500e-01, -2.5485e-01,  8.8289e-01,  7.3123e-02],
                      [ 1.1678e+00,  2.0142e+00,  1.0518e+00,  9.9192e-02],
                      [ 1.1603e+00, -6.9374e-02,  1.4627e-02,  1.24

In [28]:
%timeit Batch.cat(batches)

144 µs ± 189 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

339 µs ± 2.02 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
